# Proof-of-concept demo of Merkle Octree

This notebook demonstrates the use of an [octree](https://en.wikipedia.org/wiki/Octree) data structure whose nodes also form a [Merkle Tree](https://en.wikipedia.org/wiki/Merkle_tree).
The Merkle octree can be used to quickly detect and determine map changes to enable efficient sharing of map updates in a distributed mapping scenario.

In [1]:
from merkleoctree import MerkleOctree, DataPoint, HashTreeNode

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Create a single node tree

tree = MerkleOctree(min_bounds=[-10,-10,-10], max_bounds=[10,10,10], max_depth=8)

tree.insert(DataPoint(pos=[1,1,1], data=b'some data')) # Note: data could be any binary-encoded data (occpancy, semantic classification belief)

print('=== Before change ===')
hash_tree = tree.hash_tree

for node in hash_tree:
    print(node)

=== Before change ===
(7, 0, 0, b'\xdayr\x88r\xaa<\x1bq\xb2\xd6v\xac\xa7A?*\xe0\xac\x8f')
(6, 0, 7, b'\xac\x9c\x07\xfd\x144\x05?RE)<\xaf\x87G[(\xa5H\xcb')
(5, 7, 0, b'0f\x8f\xf6}_\xdbJ\xd2\x99\x0cn\xcb\x06\x1c0\xfeC\xed\xec')
(4, 0, 0, b'\xbc\x00\x83hk%\xd9<\xb8\xd9\x88\x86\xd8\xa4\xa9\\E/\x10a')
(3, 0, 0, b"3\x1b\x9a'I\x0b\xfc\x1d?;\xe3\x9b\x11\xe1\xea$\x86]\nL")
(2, 0, 7, b'\xd3/\xc2nq\xd4\x81!\x89l\xf5\re\xef\xd7\xba-I\x10V')
(1, 7, 7, b"F\xed\xa4\xfc7\x9a\x15\xb2O\x99\xac[\xcd\x94'\x9f\xe0I<\xd1")
(0, 7, 0, b'\xba\xf3EQ\xfe\xcbH\xac\xc3\xda\x86\x8e\xb8^\x1bm\xac\x9d\xe3V')


In [3]:
# We modify the existing data point by inserting different data to the same location.

# Note that all the hashes change, indicating at each level that a child value has changed.
# A fast change detection for the whole tree can be done by comparing only the top hash.

tree.insert(DataPoint([1,1,1], b'new data'))

print('=== After 1st change ===')

hash_tree = tree.hash_tree

for node in hash_tree:
    print(node)

=== After 1st change ===
(7, 0, 0, b'\xe4\x12\xe0)Wh\xe5\x9a\xfa\x10^\xd8\xf4z\x87$\x928\xfa~')
(6, 0, 7, b'e\x1c(\xfc\x83\xbc\x90\x87\x8e\xef\xff\x82\xbd/!\xe5\x1f\x14\x12d')
(5, 7, 0, b'\x9f\xf4\xf9\xe9?&\xc2\xbf\xaf\xb1\xd6\xa2\xba1\xc6!\x1e\xd0\xbd\x8b')
(4, 0, 0, b'\xa5,\xdb:i\x05\xc6\xdcYy\xb4>\t\xb2\xd6J}\x9a33')
(3, 0, 0, b'\xde\xa3\xc3\xf6r8d\xe1yc\xd9C\xa6\xa4\x15\x83\xb4\x96+q')
(2, 0, 7, b'\x0f\x0e\xa3i{\xd5\x1a\xe6\xe7\x81\x92\xf3\xb4\xa8\x04b\x08L\x92\x85')
(1, 7, 7, b"k?\x88h\x94,*\x92\xd2L\x85'\xc9\xa6\xad\x98\xcc\xd9\xcb-")
(0, 7, 0, b"\x01\x97\xdf\xb0A\xa9>\x9d\x19\x9f\x88\xef\xd7\x11@\xf3'_\xc6\x9d")


In [4]:
# We insert a new data point in a different location.

# Note that more hashes are shown, indicating the tree expanding to accomodate the new point.
# The space-partitioning tree structure is "right-sized" for the sparse data, and does not allocate unneeded empty voxels.

# Note that the top hash changes indicating a change in the tree.
# However hashes in the branch starting with (6, 0, 7, ...) remain the same, indicating no change in that branch, allowing 
# it to be skipped in a map update.

tree.insert(DataPoint([-1,1,-1], b'more data'))

print('=== After 2nd change ===')

hash_tree = tree.hash_tree

for node in hash_tree:
    print(node)

=== After 2nd change ===
(7, 0, 0, b'w\xcbUH\x93GD\t\xbc\xc0\x8a\xbc\xa6\x8e/\x87\x05\xba\xc4y')
(6, 0, 7, b'e\x1c(\xfc\x83\xbc\x90\x87\x8e\xef\xff\x82\xbd/!\xe5\x1f\x14\x12d')
(5, 7, 0, b'\x9f\xf4\xf9\xe9?&\xc2\xbf\xaf\xb1\xd6\xa2\xba1\xc6!\x1e\xd0\xbd\x8b')
(4, 0, 0, b'\xa5,\xdb:i\x05\xc6\xdcYy\xb4>\t\xb2\xd6J}\x9a33')
(3, 0, 0, b'\xde\xa3\xc3\xf6r8d\xe1yc\xd9C\xa6\xa4\x15\x83\xb4\x96+q')
(2, 0, 7, b'\x0f\x0e\xa3i{\xd5\x1a\xe6\xe7\x81\x92\xf3\xb4\xa8\x04b\x08L\x92\x85')
(1, 7, 7, b"k?\x88h\x94,*\x92\xd2L\x85'\xc9\xa6\xad\x98\xcc\xd9\xcb-")
(0, 7, 0, b"\x01\x97\xdf\xb0A\xa9>\x9d\x19\x9f\x88\xef\xd7\x11@\xf3'_\xc6\x9d")
(6, 0, 2, b'\x01I\xe7\xb0\xffQ\x8c\xac\xfa\xc0\xe6\x92\x1bx\x04\xa0\xc8=\x94\xcb')
(5, 2, 5, b"\xb4\x82\x99'\xf0\xa2\x89\xd2pN\xdb\xd8\xe4\xc2\x9b\xb0\x9b\x07\xbe\xf7")
(4, 5, 5, b'O\x17\x98\xdc"\xf66+\xb4\xd5\x9f\xd3\xb0XB\x97y@\x82\x98')
(3, 5, 5, b"\xc8\xdd\x17\xb0/\xe2\xe04o\nb'\xc5\xe0\xb5\x96\xbfJD\xb8")
(2, 5, 2, b'\x8a\x81\xf3\xf7}\x87\xd0\xbb\xae\xaf\x03\x08U\x

In [13]:
# We demonstrate how the hash tree might be shared independent of the underlying map data.

# Encode the hash tree as a list (can be further packed into binary with compression)
hash_tree = list(tree.hash_tree)
print('=== Hashes as list ===')
print(hash_tree)

# Recreate a hash-only Merkle tree from the encoded hash list
# Note that the reconstructed hashes are the same as the original (i.e. after 2nd change above).
hash_only = HashTreeNode.create_from_encoded(hash_tree)

print('=== Hash-only Reconstruction ===')
for node in hash_only:
    print(node)

=== Hashes as list ===
[(7, 0, 0, b'w\xcbUH\x93GD\t\xbc\xc0\x8a\xbc\xa6\x8e/\x87\x05\xba\xc4y'), (6, 0, 7, b'e\x1c(\xfc\x83\xbc\x90\x87\x8e\xef\xff\x82\xbd/!\xe5\x1f\x14\x12d'), (5, 7, 0, b'\x9f\xf4\xf9\xe9?&\xc2\xbf\xaf\xb1\xd6\xa2\xba1\xc6!\x1e\xd0\xbd\x8b'), (4, 0, 0, b'\xa5,\xdb:i\x05\xc6\xdcYy\xb4>\t\xb2\xd6J}\x9a33'), (3, 0, 0, b'\xde\xa3\xc3\xf6r8d\xe1yc\xd9C\xa6\xa4\x15\x83\xb4\x96+q'), (2, 0, 7, b'\x0f\x0e\xa3i{\xd5\x1a\xe6\xe7\x81\x92\xf3\xb4\xa8\x04b\x08L\x92\x85'), (1, 7, 7, b"k?\x88h\x94,*\x92\xd2L\x85'\xc9\xa6\xad\x98\xcc\xd9\xcb-"), (0, 7, 0, b"\x01\x97\xdf\xb0A\xa9>\x9d\x19\x9f\x88\xef\xd7\x11@\xf3'_\xc6\x9d"), (6, 0, 2, b'\x01I\xe7\xb0\xffQ\x8c\xac\xfa\xc0\xe6\x92\x1bx\x04\xa0\xc8=\x94\xcb'), (5, 2, 5, b"\xb4\x82\x99'\xf0\xa2\x89\xd2pN\xdb\xd8\xe4\xc2\x9b\xb0\x9b\x07\xbe\xf7"), (4, 5, 5, b'O\x17\x98\xdc"\xf66+\xb4\xd5\x9f\xd3\xb0XB\x97y@\x82\x98'), (3, 5, 5, b"\xc8\xdd\x17\xb0/\xe2\xe04o\nb'\xc5\xe0\xb5\x96\xbfJD\xb8"), (2, 5, 2, b'\x8a\x81\xf3\xf7}\x87\xd0\xbb\xae\xaf

In [ ]:
# TODO: Show method of computing patch update from two overlapping hash trees.